In [1]:
import geopandas as gpd
from geopandas import GeoDataFrame
import os
import zipfile
import pandas as pd
import glob
import zipfile
from pandas import DataFrame,Series
import pyproj
layer_crs = 'epsg:2039'
geodesic = pyproj.Geod(ellps='GRS80')

project_folder = os.path.dirname(os.getcwd())
# The network to work with
res_path = 'output/streets_elements'
clean_network = gpd.read_file(r'{}/streets_elements.shp'.format(res_path))
col_to_leave = ['oidrechov', 'length', 'geometry']
key_fields = 'oidrechov'
main_path = 'output/streets_elements/elements/parks_resDen_bikes'
res_path = 'output/streets_elements/detailed_folder/parks_resDen_bikes'

In [2]:
# Bike trails

In [3]:
# Upload bike data and network
name= 'bike_trails'
spec_path = f'{res_path}/{name}'
os.makedirs(spec_path,exist_ok = True)

df_elements = gpd.read_file('{}/Bicycle Routes.shp'.format(main_path)).set_index('oidshvil')
# For Azimuth calculation convrt to wgs84
df_elements_wgs84 =df_elements.to_crs('epsg:4326')

clean_network_temp = clean_network[col_to_leave].set_index(key_fields)
clean_network_temp['is_bike'] = 0
clean_network_temp['ratio'] = -1
clean_network_temp['angle'] = -200
clean_network_temp['buf_name'] = -1

In [4]:
# Buffer around each bke trail
buffers_df = df_elements.buffer(cap_style=2,distance=25,join_style=3)
buffers_df.to_file(f'{spec_path}/buffer.shp')


In [5]:
# sjoin
sjoin = clean_network_temp.sjoin(GeoDataFrame(geometry=buffers_df,crs=layer_crs),lsuffix='')
sjoin_wgs84 =sjoin.to_crs('epsg:4326')


In [6]:
def calculate_azamith(cur_str):
    cors = cur_str.geometry.coords.xy
    x_1,y_1,x_2,y_2 = cors[0][0], cors[1][0], cors[0][-1], cors[1][-1]
    return geodesic.inv(x_1,y_1,x_2,y_2)

In [7]:

for group in sjoin_wgs84.reset_index().groupby('oidrechov'):
    par_name = group[0]
    print(par_name)
    for street_temp in group[1].iterrows():
        street = street_temp[1]
        buffer_name = street['index_right']
        fwd_azimuth_0,fwd_azimuth_1,_= calculate_azamith(street)
        bike_trail = df_elements_wgs84.loc[buffer_name]
        bike_azimuth_0,_,_= calculate_azamith(bike_trail)
        clean_network_temp.at[par_name,'buf_name'] =buffer_name
        if abs(fwd_azimuth_0 -bike_azimuth_0) <30 or abs(fwd_azimuth_1- bike_azimuth_0) <30:
            street2_0 = sjoin.loc[par_name]
            if isinstance(street2_0, Series):
                street2 = street2_0
            else:
                street2 = street2_0[street2_0['index_right']==buffer_name]
            street2_poly = street2.geometry.buffer(cap_style=2,distance=1,join_style=3)
            p = street2_poly.intersection(buffers_df.loc[buffer_name])
            ratio = p.area/street2_poly.area
            clean_network_temp.at[par_name,'ratio'] = ratio
            clean_network_temp.at[par_name,'angle'] =bike_azimuth_0
            if isinstance(ratio, Series):
                ratio = ratio.values[0]
            if ratio >0.5:
                clean_network_temp.at[par_name,'is_bike'] = 1
                break
        else:
            clean_network_temp.at[par_name,'angle'] =-300
clean_network_temp.to_file('{}/streets_elements_{}.shp'.format(spec_path,name))
print('finish')

6.0
11.0
15.0
19.0
20.0
21.0
22.0
23.0
24.0
27.0
29.0
31.0
34.0
35.0
38.0
41.0
45.0
46.0
51.0
55.0
67.0
68.0
70.0
80.0
81.0
88.0
89.0
102.0
103.0
126.0
127.0
129.0
133.0
161.0
202.0
203.0
207.0
214.0
215.0
216.0
220.0
226.0
227.0
228.0
229.0
230.0
233.0
250.0
251.0
252.0
253.0
254.0
255.0
256.0
257.0
258.0
259.0
266.0
271.0
278.0
286.0
288.0
291.0
292.0
293.0
302.0
306.0
307.0
309.0
310.0
311.0
312.0
313.0
314.0
315.0
316.0
317.0
320.0
321.0
339.0
340.0
342.0
389.0
390.0
391.0
396.0
397.0
409.0
410.0
411.0
412.0
413.0
414.0
415.0
420.0
424.0
425.0
429.0
430.0
432.0
437.0
438.0
448.0
449.0
454.0
458.0
460.0
461.0
462.0
463.0
464.0
465.0
467.0
476.0
496.0
508.0
510.0
515.0
545.0
554.0
557.0
582.0
586.0
588.0
589.0
598.0
602.0
606.0
611.0
617.0
618.0
619.0
620.0
621.0
625.0
626.0
627.0
628.0
629.0
630.0
631.0
632.0
633.0
634.0
635.0
636.0
637.0
641.0
642.0
644.0
646.0
656.0
659.0
660.0
661.0
662.0
692.0
693.0
694.0
695.0
696.0
698.0
699.0
700.0
701.0
702.0
703.0
704.0
706.0
710.0
711.0
71

C:\Users\Achituv\AppData\Local\Temp\ipykernel_5264\3382149604.py:33: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  clean_network.to_file('output/streets_elements/streets_elements_temp.shp')


finish


In [70]:
## Parks

In [8]:
name = 'parks'
spec_path = f'{res_path}/{name}'
df_elements = gpd.read_file('{}/Green Areas and Public Parks.shp'.format(main_path))
os.makedirs(spec_path,exist_ok = True)
clean_network_temp = clean_network[col_to_leave].set_index(key_fields)

for buffer in range(200,550,50):
    print(buffer)
    cur_field = f'count{buffer}'
    clean_network_temp[cur_field] = 0

    # Buffer around each segment
    buffers_df = clean_network_temp.buffer(distance=buffer,join_style=2)
    buffers_df.to_file(f'{spec_path}/buffer_{buffer}.shp')
    # sjoin (by intersection),then
    clean_network_temp[cur_field] =GeoDataFrame(geometry=buffers_df,crs=layer_crs).sjoin(df_elements,how='left',lsuffix='').dissolve(by = key_fields,aggfunc='count')['index_right']

clean_network_temp.to_file('{}/streets_elements_{}.shp'.format(spec_path,name))
print('finish')

200
250
300
350
400
450
500


C:\Users\Achituv\AppData\Local\Temp\ipykernel_5264\411111905.py:22: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  clean_network.to_file('output/streets_elements/streets_elements_temp.shp')


finish
